# Solitons

## Algorithm

1. Set parameters of the equation: $\epsilon$, $\mu$.
1. Set $\Delta x$, $\Delta t$, $N_{x}$ and $N_{t}$, where $N_{x}$ and $N_{t}$ are the number of steps in space and time, respectively. If a spatial length $L$ is given, then the number of points in space are $N_{x} = \text{trunc}(L/\Delta x) + 1$ so we start from $x = 0$ up to $x = L$.
1. Since we are fixing the boundaries, the first and last point have fixed values $u_{0}^{n}$, $u_{N_{x}}^{n}$ and must be ignored from evolving them. This means that we may save memory by evolving only the inner spatial points. Thus, we create a 2-dimensional array which has one of its lengths as $N_{x} - 1$.
1. Insert some initial profile for which we have $u_{j}^{1}$ for all $j$.
1. For the first time step we obtain $u_{j}^{2}$ for all $1 \leq j \leq N_{x}-1$ as  
    1. Update the array spatial boundaries using
        \begin{align}
            u_{1}^{2} & = u_{1}^{1} - \frac{\epsilon}{6}\frac{\Delta t}{\Delta x}[u_{2}^{1} + u_{1}^{1} + u_{0}^{1}][u_{2}^{1} - u_{0}^{1}] - \frac{\mu}{2}\frac{\Delta t}{\Delta x^{3}}[u_{3}^{1} - 2 u_{2}^{1} + u_{0}^{1}] \\
            u_{N_{x}-1}^{2} & = u_{N_{x}-1}^{1} - \frac{\epsilon}{6}\frac{\Delta t}{\Delta x}[u_{N_{x}}^{1} + u_{N_{x}-1}^{1} + u_{N_{x}-2}^{1}][u_{N_{x}}^{1} - u_{N_{x}-2}^{1}] - \frac{\mu}{2}\frac{\Delta t}{\Delta x^{3}}[-u_{N_{x}}^{1} + 2 u_{N_{x}-2}^{1} - u_{N_{x}-3}^{1}]
        \end{align}
    1. Update the rest of the points according to 
         \begin{equation}
             u_{j}^{2} = u_{j}^{1} - \frac{\epsilon}{6} \frac{\Delta t}{\Delta x} \left[ u_{j+1}^{1} + u_{j}^{1} + u_{j-1}^{1} \right] \left[u_{j+1}^{1} - u_{j-1}^{1}\right] - \frac\mu2 \frac{\Delta t}{\Delta x^3} \left[ u_{j+2}^{1} + 2u_{j-1}^{1} - 2u_{j+1}^{1} - u_{j-2}^{1} \right]
         \end{equation}
1. For the remaining steps. Update according to:
    1. First change the array spatial boundaries with
        \begin{align}
            u^{n+1}_{1} & = u^{n-1}_{1} - \frac{\epsilon}{3} \frac{\Delta t}{\Delta x} \left[ u_{2}^{n} + u_{1}^{n} + u_{0}^{n} \right]\left[u_{2}^{n} - u_{0}^{n}\right] - \mu \frac{\Delta t}{\Delta x^3} \left[ u_{3}^{n} + u_{0}^{n} - 2u_{2}^{n} \right] \\
            u^{n+1}_{N_{x}-1} & = u^{n-1}_{N_{x}-1} - \frac{\epsilon}{3} \frac{\Delta t}{\Delta x} \left[ u_{N_{x}}^{n} + u_{N_{x}-1}^{n} + u_{N_{x}-2}^{n} \right]\left[u_{N_{x}}^{n} - u_{N_{x}-2}^{n}\right] - \mu \frac{\Delta t}{\Delta x^3} \left[2u_{N_{x}-2}^{n} - u_{N_{x}}^{n} - u_{N_{x}-3}^{n} \right]
        \end{align}
    1. Then change the rest of the points with:
        \begin{equation}
            u^{n+1}_j = u^{n-1}_j - \frac{\epsilon}{3} \frac{\Delta t}{\Delta x} \left[ u_{j+1}^{n} + u_{j}^{n} + u_{j-1}^{n} \right]\left[u_{j+1}^{n} - u_{j-1}^{n}\right] - \mu \frac{\Delta t}{\Delta x^3} \left[ u_{j+2}^{n} + 2u_{j-1}^{n} - 2u_{j+1}^{n} - u_{j-2}^{n} \right]
        \end{equation}
        
        
Since at each time step after the first we require the two previous profiles in time and we don't want/need the profile at each step, we opt for saving a 2-dimensional array with time length equal to $3$

## Functions

In [1]:
using Plots

function kdev(initial_u::Function, Δx::Float64, Δt::Float64, nx::Int, nt::Int, ϵ::Float64, μ::Float64, smpl)
    u = zeros(nx-1,3)
    u[:,1] = [initial_u(x) for x in range(Δx, step=Δx, length=nx-1)]
    u0 = initial_u(0)
    un = initial_u(nx*Δx)
    
    samples = trunc(Int, nt/smpl) + 1
    
    u_samples = zeros(nx-1, samples)
        
    if mod(nt, smpl) == 0
        j = samples - (nt÷smpl)
        u_samples[:,j] = u[:,1]
    end
    
    a = ϵ*Δt/(3*Δx)
    b = μ*Δt/(Δx^3)
    
    # First step
    
    u[1,2] = u[1,1] - (a/2)*(u[2,1]+u[1,1]+u0)*(u[2,1]-u0) - (b/2)*(u[3,1]-2*u[2,1]+u0)
    u[2,2] = u[2,1] - (a/2)*(u[3,1]+u[2,1]+u[1,1])*(u[3,1]-u[1,1]) - (b/2)*(u[4,1]-2*u[3,1]+2*u[1,1]-u0)
    u[end-1,2] = (u[end-1,1] - (a/2)*(u[end,1]+u[end-1,1]+u[end-2,1])*(u[end,1]-u[end-2,1]) 
                    - (b/2)*(un-2*u[end,1]+2*u[end-2,1]-u[end-3,1]))
    u[end,2] = u[end,1] - (a/2)*(un+u[end,1]+u[end-1,1])*(un-u[end-1,1])- (b/2)*(-un+2*u[end-1,1]-u[end-2,1])
        
    for i in range(3, nx-3)
        u[i,2] = (u[i,1] - (a/2)*(u[i+1,1]+u[i,1]+u[i-1,1])*(u[i+1,1]-u[i-1,1]) 
                    - (b/2)*(u[i+2,1]-2*u[i+1,1]+2*u[i-1,1]-u[i-2,1]))
    end
    
    if mod(nt-1, smpl) == 0
        j = samples - ((nt-1)÷smpl)
        u_samples[:,j] = u[:,2]
    end
    
    # Every other step
    
    for k in range(1, nt-1)
        n = mod1(k,3)
        n1 = mod1(k+1,3)
        n2 = mod1(k+2,3)
        
        u[1,n2] = u[1,n] - a*(u[2,n1]+u[1,n1]+u0)*(u[2,n1]-u0)-b*(u[3,n1]-2*u[2,n1]+u0)
        u[2,n2] = u[2,n] - a*(u[3,n1]+u[2,n1]+u[1,n1])*(u[3,n1]-u[1,n1]) - b*(u[4,n1]-2*u[3,n1]+2*u[1,n1]-u0)
        u[end-1,n2] = (u[end-1,n] - a*(u[end,n1]+u[end-1,n1]+u[end-2,n1])*(u[end,n1]-u[end-2,n1]) 
                        - b*(un+2*u[end-2,n1]-2*u[end,n1]-u[end-3,n1])
                       )
        u[end,n2] = u[end,n] - a*(un+u[end,n1]+u[end-1,n1])*(un-u[end-1,n1])- b*(2*u[end-1,n1]-un-u[end-2,n1])
                    
        for i in range(3, nx-3)
            u[i,n2] = (u[i,n] - a*(u[i+1,n1]+u[i,n1]+u[i-1,n1])*(u[i+1,n1]-u[i-1,n1]) 
                        - b*(u[i+2,n1]+2*u[i-1,n1]-2*u[i+1,n1]-u[i-2,n1]))
        end
        
        if mod(nt-(k+1),smpl) != 0
            continue
        end
        
        j = samples - ((nt-(k+1))÷smpl)
        u_samples[:,j] = u[:,n2]
    end
    
    return u_samples
end


# Make the animation
@userplot SolitonPlot
@recipe function s(sol::SolitonPlot)
    times, x, u, i, my_ylims = sol.args
    t = round(Int, times[i])
    label --> false
    title --> "\$ t=$t \$"
    xlabel --> "\$ x \$"
    ylabel --> "\$ u(x,t) \$"
    ylims --> my_ylims
    labelfontsize --> 13
    linewidth --> 2.
    color --> :blue
    background_color --> :black
    dpi --> 300
    x, u[:,i]
end

# Part b)
## Simple surface plot of u(x,t) at the sampled times

In [ ]:
initial_u(x) = 0.5*(1-tanh((x-25)/5))
Δx = 0.4
Δt = 0.1
nx = 130
nt = 2000
ϵ = 0.2
μ = 0.1
smpl = 1

x = range(Δx, step=Δx, length=nx-1)
t_values = [steps*Δt for steps in range(0, step=1, length=nt+1) if mod(nt-steps,smpl) == 0]
u_profiles = kdev(initial_u, Δx, Δt, nx, nt, ϵ, μ, smpl);

In [ ]:
n=size(u_profiles)[2]
max_magnitudes = [maximum(abs.(u_profiles[:,i])) for i ∈ 1:n]
stability = (Δt/Δx)*(ϵ*max_magnitudes .+ (4*μ/(Δx^2)))

stability_plot = plot(t_values, stability, xlabel="\$ t \$", legendfontsize=14, lw=1.6, dpi=500, legend=:bottomright,
     label="\$ \\frac{\\Delta t}{\\Delta x}\\left(\\epsilon | u | + 4\\frac{\\mu}{(\\Delta x)^{2}} \\right) \$")
# savefig(stability_plot, "stability_plot.png")

In [ ]:
plot_part_b = surface(x, t_values, u_profiles', xlabel="\$ x \$", ylabel="\$ t \$", zlabel="\$ u(x,t) \$", 
    camera=(35,55), c = :blues, colorbar=:false, dpi=500, background_color= RGB(225/255,193/255,177/255))
# savefig(plot_part_b, "solitons_b_sand.png")

## 2-dimensional plots of part b)

In [ ]:
colors = ["gray", "blue", "magenta", "red", "indigo", "hotpink"]

u_plot1 = plot(xlabel="\$ x \$", legend=:bottomleft, ylabel="\$ u(x, t) \$")

my_width = [1.3 + i*0.05 for i in range(1, 6)]

alpha_values = [0.4 + 0.2*i for i in range(1, 6)]

for (i, j) in enumerate([1, 3, 5, 7, 9])
    time = round(Int, t_values[j])
    plot!(x, u_profiles[:,j], label="\$ t = $time \$", color=colors[i], alpha=alpha_values[i], 
        linewidth=my_width[i], dpi=500)
end

u_plot1
# savefig(u_plot1, "solitons_b_2d.png")

## Gif for part b)

### First generate the data

In [2]:
initial_u(x) = 0.5*(1-tanh((x-25)/5))
Δx = 0.4
Δt = 0.1
nx = 130
nt = 2000
ϵ = 0.2
μ = 0.1
smpl = 8

x = range(Δx, step=Δx, length=nx-1)
t_values = [steps*Δt for steps in range(0, step=1, length=nt+1) if mod(nt-steps,smpl) == 0]
u_profiles = kdev(initial_u, Δx, Δt, nx, nt, ϵ, μ, smpl);

### Then make the animation

In [ ]:
n=size(u_profiles)[2]
ymax = maximum(u_profiles)
ylims = (0, ymax)

anim = @animate for i ∈ 1:n
    solitonplot(t_values, x, u_profiles, i, ylims)
end

anim
gif(anim, "soliton_b_50.gif", fps = 50)

# Part c)

## Trying to adjust the sum of solitons with the initial positions and amplitudes as specified

In [38]:
Δx = 0.4
Δt = 0.1
nx = 130
nt = 5500
ϵ = 0.2
μ = 0.1
smpl = 10

# First soliton
x1_0 = 12.  # Initial position of the peak of the soliton
c1 = 4/75
soliton1(x) = (3*c1/ϵ)*(sech(0.5*sqrt(c1/μ)*(x-x1_0)))^2.

# Second soliton
x2_0 = 26.  # Initial position of the peak of the soliton
c2 = 3/150
soliton2(x) = (3*c2/ϵ)*(sech(0.5*sqrt(c2/μ)*(x-x2_0)))^2.

# Initial u(x,t) is a sum of two solitons
initial_u(x) = soliton1(x) + soliton2(x)

x = range(Δx, step=Δx, length=nx-1)
t_values = [steps*Δt for steps in range(0, step=1, length=nt+1) if mod(nt-steps,smpl) == 0]
u_profiles = kdev(initial_u, Δx, Δt, nx, nt, ϵ, μ, smpl);

### Plot of the initial and final configurations u(x,t)

In [34]:
t0 = round(Int, t_values[1])
tf = round(Int, t_values[end])
ymax = maximum(u_profiles)
my_ylims = (0, ymax)

solitons_c = plot(x, u_profiles[:,1], label = "\$ t=$t0 \$", xlabel="\$ x \$", ylabel="\$ u(x,t) \$", labelfontsize=13, 
    legendfontsize=13, ylims=my_ylims, linewidth=2, background_color=:black, color=:blue, dpi=300, 
    legend=:top)
plot!(x, u_profiles[:,end], label = "\$ t=$tf \$", color=:red, linewidth=2)
# savefig(solitons_c, "solitons_c.png")

### Gif

In [ ]:
n=size(u_profiles)[2]
ymax = maximum(u_profiles)
ylims = (0, ymax)

anim = @animate for i ∈ 1:n
    solitonplot(t_values, x, u_profiles, i, ylims)
end
anim
# gif(anim, "solitons_c_50.gif", fps = 50)

In [48]:
n=size(u_profiles)[2]
max_magnitudes = [maximum(abs.(u_profiles[:,i])) for i ∈ 1:n]
stability = (Δt/Δx)*(ϵ*max_magnitudes .+ (4*μ/(Δx^2)))

stability_plot = plot(t_values, stability, xlabel="\$ t \$", legendfontsize=9, lw=1.6, dpi=500, 
                      legend=:bottomright,
label="\$ \\frac{\\Delta t}{\\Delta x}\\left(\\epsilon |u(x,t)| + 4\\frac{\\mu}{(\\Delta x)^{2}} \\right) \$")
savefig(stability_plot, "stability_plot_c.png")

## My own parameters

In [ ]:
Δx = 0.4
Δt = 0.1
nx = 130
nt = 2000
ϵ = 0.2
μ = 0.1
smpl = 8

# First soliton
x1_0 = 12.  # Initial position of the peak of the soliton
x1_final = 47.  # Final position at which we want to observe the soliton
c1 = (x1_final - x1_0)/(nt*Δt)
soliton1(x) = (3*c1/ϵ)*(sech(0.5*sqrt(c1/μ)*(x-x1_0)))^2.

# Second soliton
x2_0 = 26.  # Initial position of the peak of the soliton
x2_final = 35.  # Final position at which we want to observe the soliton
c2 = (x2_final - x2_0)/(nt*Δt)
soliton2(x) = (3*c2/ϵ)*(sech(0.5*sqrt(c2/μ)*(x-x2_0)))^2.

# Initial u(x,t) is a sum of two solitons
initial_u(x) = soliton1(x) + soliton2(x)

x = range(Δx, step=Δx, length=nx-1)
t_values = [steps*Δt for steps in range(0, step=1, length=nt+1) if mod(nt-steps,smpl) == 0]
u_profiles = kdev(initial_u, Δx, Δt, nx, nt, ϵ, μ, smpl);

In [ ]:
n=size(u_profiles)[2]
ymax = maximum(u_profiles)
ylims = (0, ymax)

anim = @animate for i ∈ 1:n
    solitonplot(t_values, x, u_profiles, i, ylims)
end
anim
# gif(anim, "soliton2_c_50.gif", fps = 50)